## PIP Installs

In [1]:
'''
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git
!pip install openpyxl
!git clone https://github.com/amitt001/Android-App-Reviews-Dataset.git
!pip install spacy==2.2.3
!pip install beautifulsoup4==4.9.1
!pip install textblob==0.15.3
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
'''

'\n!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git\n!pip install openpyxl\n!git clone https://github.com/amitt001/Android-App-Reviews-Dataset.git\n!pip install spacy==2.2.3\n!pip install beautifulsoup4==4.9.1\n!pip install textblob==0.15.3\n!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git\n!pip install -U pip setuptools wheel\n!pip install -U spacy\n!python -m spacy download en_core_web_sm\n'

## Imports

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import preprocess_kgptalkie as ps
import re
import pandas as pd
import numpy as np
import pickle
import os

## Read Dataset

In [3]:
# Read negative sentiment text file
df_dataset1 = pd.read_csv('Android-App-Reviews-Dataset/negative10k.txt', header=None, names=['Reviews'], sep='\t', lineterminator='\n')
df_dataset1 = df_dataset1.assign(Sentiment='negative')

# Read positive sentiment text file
df_dataset2 = pd.read_csv('Android-App-Reviews-Dataset/positive10k.txt', header=None, names=['Reviews'], sep='\t', lineterminator='\n')
df_dataset2 = df_dataset2.assign(Sentiment='positive')

#Merge two datasets
df_dataset = pd.concat([df_dataset1,df_dataset2])


## Clean Function

In [4]:
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    #x = ps.remove_html_tags(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

df_dataset['Reviews'] = df_dataset['Reviews'].apply(lambda x: get_clean(x))
df_dataset.head()

,Reviews,Sentiment
0,ya no i would your going make an application m...,negative
1,stupid adds the stupid adds pop up every 5 sec...,negative
2,what the fuck needs way more detail 1 sun shou...,negative
3,i love this game but all of a sudden it when i...,negative
4,mark now i cannot send photo and images pls fi...,negative


## Model Training

In [5]:
tfidf = TfidfVectorizer(max_features=5000)
X = df_dataset['Reviews']
y = df_dataset['Sentiment']

X = tfidf.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

clf = LinearSVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

## Classification Score

In [6]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.88      0.88      0.88      1940
    positive       0.88      0.89      0.89      1997

    accuracy                           0.88      3937
   macro avg       0.88      0.88      0.88      3937
weighted avg       0.88      0.88      0.88      3937



## Save Model to File

In [7]:
pickle.dump(clf, open('model.pkl', 'wb'))
pickle.dump(tfidf, open('tfidf.pkl', 'wb'))

In [2]:
folders = ['apps_made_for_disabled/','browser/','entertainment/','music/','shopping/','social_media/']

In [6]:
num_reviews = 0
for folder in folders:
    files = os.listdir(folder)
    for file in files:
        if '.csv' in file:
            df = pd.read_csv(os.path.join(folder,file))
            num_reviews += df.shape[0]


/tmp/ipykernel_14407/2248178964.py:6: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder,file))
/tmp/ipykernel_14407/2248178964.py:6: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder,file))
/tmp/ipykernel_14407/2248178964.py:6: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder,file))
/tmp/ipykernel_14407/2248178964.py:6: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder,file))


In [7]:
num_reviews

11002019